In [1]:
HOST = "localhost"
PORT = "8124"
TABLE_NAME = "KnowledgeBase"
MODEL_EMB_NAME = "ai-forever/sbert_large_nlu_ru"
MODEL_CHAT_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# MODEL_CHAT_NAME = "Vikhrmodels/Vikhr-7B-instruct_0.4"
SYSTEM_PROMPT = """
INSTRUCT:
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.

If you receive a question that is harmful, unethical, or inappropriate, end the dialogue immediately and do not provide a response. 

If you make a mistake, apologize and correct your answer.

Generate a response based solely on the provided document.

Answer the following question language based only on the CONTEXT provided.

Отвечай только на русском языке.
"""

In [2]:
import clickhouse_connect, torch
from transformers import AutoModel, AutoTokenizer, Conversation, pipeline
from typing import List, Union

In [3]:
def search_results(connection, table_name: str, vector: list[float], limit: int = 5):
    """
    Поиск результатов похожих векторов в базе данных.

    Parameters:
    - connection (Connection): Соединение с базой данных.
    - table_name (str): Название таблицы, содержащей вектора и другие данные.
    - vector (List[float]): Вектор для сравнения.
    - limit (int): Максимальное количество результатов.

    Returns:
    - List[dict]: Список результатов с наименованием, URL, датой, номером, текстом и расстоянием.

    Examples:
    >>> connection = Connection(...)
    >>> vector = [0.1, 0.2, 0.3]
    >>> results = search_results(connection, 'my_table', vector, limit=5)
    """
    res = []
    # Инициализируем список результатов
    vector = ",".join([str(float(i)) for i in vector])
    # Выполняем запрос к базе данных
    with connection.query(
        f"""SELECT Id, Source, BusinessLineId, Direction, Product, Type, Description, Title, Url, ParentTitle, ParentUrl, ChunkType, cosineDistance(({vector}), Embedding) as score FROM {table_name} ORDER BY score ASC LIMIT {limit+500}"""
    ).rows_stream as stream:
        for item in stream:
            id, source, business_line_id, direction, product, type, description, title, url, parent_title, parent_url, chunk_type, score = item

            # Добавляем результат в список
            res.append(
                {
                    "id": id,
                    "source": source,
                    "business_line_id": business_line_id,
                    "direction": direction,
                    "product": product,
                    "type": type,
                    "description": description,
                    "title": title,
                    "url": url,
                    "parent_title": parent_title,
                    "parent_url": parent_url,
                    "chunk_type": chunk_type,
                    "distance": score,
                }
            )

    # Возвращаем первые limit результатов
    res = [item for item in res if len(item["description"]) > 100]
    return res[:limit]


def mean_pooling(model_output: tuple, attention_mask: torch.Tensor) -> torch.Tensor:
    """
    Выполняет усреднение токенов входной последовательности на основе attention mask.

    Parameters:
    - model_output (tuple): Выход модели, включающий токенов эмбеддинги и другие данные.
    - attention_mask (torch.Tensor): Маска внимания для указания значимости токенов.

    Returns:
    - torch.Tensor: Усредненный эмбеддинг.

    Examples:
    >>> embeddings = model_output[0]
    >>> mask = torch.tensor([[1, 1, 1, 0, 0]])
    >>> pooled_embedding = mean_pooling((embeddings,), mask)
    """
    # Получаем эмбеддинги токенов из выхода модели
    token_embeddings = model_output[0]

    # Расширяем маску внимания для умножения с эмбеддингами
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )

    # Умножаем каждый токен на его маску и суммируем
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

    # Суммируем маски токенов и обрезаем значения, чтобы избежать деления на ноль
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Вычисляем усредненный эмбеддинг
    return sum_embeddings / sum_mask


def txt2embeddings(
    text: Union[str, List[str]], tokenizer, model, device: str = "cpu"
) -> torch.Tensor:
    """
    Преобразует текст в его векторное представление с использованием модели transformer.

    Parameters:
    - text (str): Текст для преобразования в векторное представление.
    - tokenizer: Токенизатор для предобработки текста.
    - model: Модель transformer для преобразования токенов в вектора.
    - device (str): Устройство для вычислений (cpu или cuda).

    Returns:
    - torch.Tensor: Векторное представление текста.

    Examples:
    >>> text = "Пример текста"
    >>> tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    >>> model = AutoModel.from_pretrained("bert-base-multilingual-cased")
    >>> embeddings = txt2embeddings(text, tokenizer, model, device="cuda")
    """
    # Кодируем входной текст с помощью токенизатора
    if isinstance(text, str):
        text = [text]
    encoded_input = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
    )
    # Перемещаем закодированный ввод на указанное устройство
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    # Получаем выход модели для закодированного ввода
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Преобразуем выход модели в векторное представление текста
    return mean_pooling(model_output, encoded_input["attention_mask"])


def load_models(model: str, device: str = "cpu", torch_dtype: str = "auto") -> tuple:
    """
    Загружает токенизатор и модель для указанной предобученной модели.

    Parameters:
    - model (str): Название предобученной модели, поддерживаемой библиотекой transformers.

    Returns:
    - tuple: Кортеж из токенизатора и модели.

    Examples:
    >>> tokenizer, model = load_models("ai-forever/sbert_large_nlu_ru")
    """
    # Загружаем токенизатор для модели
    tokenizer = AutoTokenizer.from_pretrained(
        model, device_map=device, torch_dtype=torch_dtype
    )

    # Загружаем модель
    model = AutoModel.from_pretrained(model, device_map=device, torch_dtype=torch_dtype)

    return tokenizer, model


def load_chatbot(model: str, device: str = "cuda", torch_dtype: str = "auto"):
    """
    Загружает чатбота для указанной модели.

    Parameters:
    - model (str): Название модели для загрузки чатбота.

    Returns:
    - Conversation: Объект чатбота, готовый для использования.

    Examples:
    >>> chatbot = load_chatbot("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    """
    # Загружаем чатбот с помощью pipeline из библиотеки transformers
    chatbot = pipeline(
        model=model,
        trust_remote_code=True,
        torch_dtype=torch_dtype,
        device_map=device,
        task="conversational",
    )
    return chatbot


def append_documents_to_conversation(conversation, documents, limit: int = 3):
    if limit > len(documents):
        texts = [document["description"] for document in documents]
    else:
        texts = [document["description"] for document in documents[:limit]]

    text = "\n".join(texts)

    document_template = f"""
    CONTEXT:
    {text}
    Отвечай только на русском языке.
    
    ВОПРОС:
    """
    conversation.add_message({"role": "user", "content": document_template})

    return conversation
        

def generate_answer(
    chatbot,
    conversation: Conversation,
    max_new_tokens: int = 128,
    temperature=0.7,
    top_k: int = 50,
    top_p: float = 0.95,
    repetition_penalty: float = 2.0,
    do_sample: bool = True,
    num_beams: int = 2,
    early_stopping: bool = True,
) -> str:
    """
    Генерирует ответ от чатбота на основе предоставленного чата и, возможно, документа.

    Parameters:
    - chatbot (Conversation): Объект чатбота.
    - chat (List[Dict[str, str]]): Список сообщений в чате.
    - document (str): Документ, если он предоставлен.

    Returns:
    - str: Сгенерированный ответ от чатбота.

    Examples:
    >>> chat = [
    >>>     {"role": "user", "content": "Привет, как дела?"},
    >>>     {"role": "system", "content": "Всё отлично, спасибо!"},
    >>> ]
    >>> document = "Это документ для обработки"
    >>> answer = generate_answer(chatbot, chat, document)
    """
    # Генерируем ответ от чатбота
    conversation = chatbot(
        conversation,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=do_sample,
        num_beams=num_beams,
        early_stopping=early_stopping,
    )

    # Возвращаем последнее сообщение чатбота как ответ
    return conversation

In [4]:
tokenizer, model = load_models(MODEL_EMB_NAME, device="cpu")

In [5]:
# Создаем объект разговора
conversation = Conversation()
# Добавляем системную инструкцию
conversation.add_message({"role": "system", "content": SYSTEM_PROMPT})

In [6]:
question = "Какую онлайн-кассу выбрать, если я продаю в интернете?" #String

In [7]:
embedding = txt2embeddings(question, tokenizer, model)

In [8]:
client = clickhouse_connect.get_client(host=HOST, port=PORT)
print("Ping:", client.ping())

Ping: True


In [9]:
documents = search_results(client, TABLE_NAME, embedding[0], limit=10)

In [10]:
print(*documents, sep='\n')

{'id': UUID('80673f6d-586a-42e7-b3b7-26f439d99f36'), 'source': 'web', 'business_line_id': 'business', 'direction': 'account', 'product': 'business-card', 'type': 'card', 'description': 'Платить виртуальной картой можно в интернете. Просто введите ее реквизиты — номер, срок действия, CVC‑код — на сайте магазина. Где посмотреть реквизиты виртуальной бизнес-карты Если хотите расплачиваться бесконтактно в офлайн-магазинах, добавьте карту в приложение Mir Pay с помощью подсказок в приложении. Приложение работает только на Android.', 'title': 'Как платить виртуальной бизнес-картой?', 'url': 'https://www.tinkoff.ru/business/help/account/business-card/virtual-and-currency/virtual/?card=payment-virtual-card', 'parent_title': 'Виртуальная бизнес-карта', 'parent_url': 'https://www.tinkoff.ru/business/help/account/business-card/virtual-and-currency/virtual/', 'chunk_type': 'recursive', 'distance': 0.319852081015203}
{'id': UUID('e7fab920-1738-4f95-88f3-075b7d3ffc47'), 'source': 'web', 'business_li

In [11]:
print(*[document["url"] for document in documents], sep='\n')

https://www.tinkoff.ru/business/help/account/business-card/virtual-and-currency/virtual/?card=payment-virtual-card
https://www.tinkoff.ru/business/help/business-invest/account/use/bonds/?card=q2
https://www.tinkoff.ru/business/help/account/business-card/about/mcc-setup/?card=paid-or-free
https://www.tinkoff.ru/business/help/account/business-card/about/mcc-setup/?card=paid-or-free
https://www.tinkoff.ru/business/help/account/business-card/about/use-card/?card=which-limits
https://www.tinkoff.ru/business/help/account/business-card/about/use-card/?card=which-limits
https://www.tinkoff.ru/business/help/account/business-card/virtual-and-currency/virtual/?card=what-for-virtual-card
https://www.tinkoff.ru/business/help/account/business-card/about/use-card/?card=for-personal-use
https://www.tinkoff.ru/business/help/account/business-card/about/use-card/?card=for-personal-use
https://www.tinkoff.ru/business/help/account/business-card/about/mcc-setup/?card=limits-for-one-card


In [12]:
conversation = append_documents_to_conversation(conversation, documents, limit=3)
conversation.add_message({"role": "user", "content": question})

In [13]:
chatbot = load_chatbot(MODEL_CHAT_NAME, device="cuda")

In [14]:
torch.cuda.empty_cache()

In [15]:
conversation = generate_answer(chatbot, conversation, temperature=0.9)
print(conversation[-1]["content"])

Если вы продаете в интернете, то лучше выбрать онлайн-кассу, которая будет поддерживать ваши транзакции через Интернет. Это позволит вам повысить уровень безопасности и защитить ваши данные. Некоторые популярные онлайн-кассы, такие как PayPal, Square, Stripe, и Amazon Pay, поддерживают веб-приложения, которые позволяют вам совершать транзакции без требований к физиче


In [16]:
conversation

Conversation id: 42ab779b-216c-4185-a374-f09518a82ddf
system: 
INSTRUCT:
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.

If you receive a question that is harmful, unethical, or inappropriate, end the dialogue immediately and do not provide a response. 

If you make a mistake, apologize and correct your answer.

Generate a response based solely on the provided document.

Answer the following question language based only on the CONTEXT provided.

Отвечай только на русском языке.

user: 
    CONTEXT:
    Платить виртуальной картой можн